In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=75d76a09cf147d1825010f4d1ad2074b43505e489046ef02019078cc35851183
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TestApp").master('local').getOrCreate()

In [4]:
spark

In [5]:
!wget https://raw.githubusercontent.com/akr808/refreshers/main/spark/spark_wine_data.csv

--2023-03-25 02:42:03--  https://raw.githubusercontent.com/akr808/refreshers/main/spark/spark_wine_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2905095 (2.8M) [text/plain]
Saving to: ‘spark_wine_data.csv’

spark_wine_data.csv 100%[===================>]   2.77M  --.-KB/s    in 0.08s   

2023-03-25 02:42:04 (34.1 MB/s) - ‘spark_wine_data.csv’ saved [2905095/2905095]



In [7]:
df = spark.read.csv('spark_wine_data.csv',inferSchema=True,header=True,sep=',')

In [8]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- points: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- province: string (nullable = true)



In [9]:
df.show()

+---------+------+-----+-----------------+
|  country|points|price|         province|
+---------+------+-----+-----------------+
|    Italy|    87| null|Sicily & Sardinia|
| Portugal|    87|   15|            Douro|
|       US|    87|   14|           Oregon|
|       US|    87|   13|         Michigan|
|       US|    87|   65|           Oregon|
|    Spain|    87|   15|   Northern Spain|
|    Italy|    87|   16|Sicily & Sardinia|
|   France|    87|   24|           Alsace|
|  Germany|    87|   12|      Rheinhessen|
|   France|    87|   27|           Alsace|
|       US|    87|   19|       California|
|   France|    87|   30|           Alsace|
|       US|    87|   34|       California|
|    Italy|    87| null|Sicily & Sardinia|
|       US|    87|   12|       California|
|  Germany|    87|   24|            Mosel|
|Argentina|    87|   30|            Other|
|Argentina|    87|   13| Mendoza Province|
|    Spain|    87|   28|   Northern Spain|
|       US|    87|   32|         Virginia|
+---------+

In [10]:
from pyspark.sql.functions import sum,avg,max,count
from pyspark.sql.functions import col

In [11]:
df.groupBy('country').agg(sum('points').alias("Sum_Points"),avg('points').alias("Avg_Points"), avg('price').alias('Avg_Price')).show(50)

+--------------------+----------+-----------------+------------------+
|             country|Sum_Points|       Avg_Points|         Avg_Price|
+--------------------+----------+-----------------+------------------+
|              Turkey|      7928|88.08888888888889|24.633333333333333|
|             Germany|    194529|89.85173210161663| 42.25754716981132|
|              France|   1962855|88.84510931064138|41.139120162016205|
|              Greece|     40674|87.28326180257511|22.364425162689805|
|            Slovakia|        87|             87.0|              16.0|
|                null|      5584|88.63492063492063|24.593220338983052|
|           Argentina|    329499|86.71026315789474|24.510117145899894|
|                Peru|      1337|          83.5625|           18.0625|
|               India|       812|90.22222222222223|13.333333333333334|
|               China|        89|             89.0|              18.0|
|               Chile|    386799| 86.4935152057245|20.786458333333332|
|     

In [12]:
df.groupBy('country','province').agg(sum('points').alias("Sum_Points")).orderBy(col('Sum_Points').desc(),col('country').asc()).show()

+---------+------------------+----------+
|  country|          province|Sum_Points|
+---------+------------------+----------+
|       US|        California|   3212491|
|       US|        Washington|    768415|
|   France|          Bordeaux|    526723|
|    Italy|           Tuscany|    525273|
|       US|            Oregon|    478476|
|   France|          Burgundy|    356486|
|    Spain|    Northern Spain|    336643|
|    Italy|          Piedmont|    333883|
|Argentina|  Mendoza Province|    283402|
|    Italy|            Veneto|    238234|
|       US|          New York|    234336|
|   France|            Alsace|    218068|
|    Italy|Northeastern Italy|    188135|
|   France|      Loire Valley|    164244|
|    Italy| Sicily & Sardinia|    158332|
|   France|         Champagne|    145549|
|   France|  Southwest France|    133177|
|Australia|   South Australia|    120112|
|   France|          Provence|    118731|
|    Italy|    Southern Italy|    118494|
+---------+------------------+----

In [13]:
df.describe().show()

+-------+---------+------------------+------------------+--------+
|summary|  country|            points|             price|province|
+-------+---------+------------------+------------------+--------+
|  count|   129908|            129971|            120975|  129908|
|   mean|     null| 88.44713820775404|35.363389129985535|    null|
| stddev|     null|3.0397302029160067|41.022217668087315|    null|
|    min|Argentina|                80|                 4|  Achaia|
|    max|  Uruguay|               100|              3300|    Župa|
+-------+---------+------------------+------------------+--------+

